# Base Case: Circular Orbit of Radius 1 AU, Period 1 Year

Create synthetic data for the simplest base case: a circular orbit of radius 1.

Can think of this as approximating the earth: radius = 1 AU, period = 1 year, mass of sun $m_0$ = 1 solar mass

\begin{align}
x(t) &= \cos(\omega t) \\
y(t) &= \sin(\omega t) \\
\omega &= 2 \pi
\end{align}

Taking two derivatives
\begin{align}
\ddot{x}(t) = -\omega^2 x(t)\\
\ddot{y}(t) = -\omega^2 y(t)
\end{align}

Equating the acceleration to $G \cdot m_0$, we can see that in these units the gravitational constant $G$ is
$$G = 4 \pi^2$$

In [1]:
# Library imports
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Aliases
keras = tf.keras

# Local imports
import utils

In [2]:
# Grow GPU memory
utils.gpu_grow_memory()

In [3]:
# The angular velocity of the earth in the (AU, years) coordinate units
omega_earth = 2.0 * np.pi

# The gravitational constant in units of (AU, years, solar_mass)
G = omega_earth**2

# The mass of the sun in this coordinate system
m0 = 1.0

# The gravitational coefficient G*(m0 + m1) for the restricted two body problem in these units
mu = G * (m0)

In [4]:
def make_train(theta0):
    """Make an array of training data points over 10 years"""
    # Sample time at intervals of 365 per year (approximately daily)
    t = np.arange(3651) / 365.0
    # Number of samples
    N = t.shape[0]
    # Set the angular frequency omega to that of earth
    omega = omega_earth
    # Theta is offset by theta0
    theta = omega * t - theta0
    # Compute x and y from theta
    x = np.cos(theta)
    y = np.sin(theta)
    # Compute vx and vy
    vx = -omega * np.sin(theta)
    vy = +omega * np.cos(theta)

    # Assemble the input DataFrame
    df_input = pd.DataFrame()
    df_input['t'] = t
    df_input['x0'] = np.ones(N) * x[0]
    df_input['y0'] = np.ones(N) * y[0]
    df_input['vx0'] = np.ones(N) * vx[0]
    df_input['vy0'] = np.ones(N) * vy[0]

    # Assemble the output DataFrame
    df_output = pd.DataFrame()
    df_output['x'] = x
    df_output['y'] = y
    df_output['vx'] = vx
    df_output['vy'] = vy    
    
    # Return the DataFrames
    return (df_input, df_output)

In [5]:
df_input, df_output = make_train(0.0)

In [6]:
df_input[0:3]

,t,x0,y0,vx0,vy0
0,0.000000,1.0,0.0,-0.0,6.283185
1,0.002740,1.0,0.0,-0.0,6.283185
2,0.005479,1.0,0.0,-0.0,6.283185


In [7]:
df_output[0:365:30]

,x,y,vx,vy
0,1.000000,0.000000,-0.000000,6.283185
30,0.869589,0.493776,-3.102483,5.463791
60,0.512371,0.858764,-5.395773,3.219325
90,0.021516,0.999769,-6.281731,0.135190
120,-0.474951,0.880012,-5.529280,-2.984206
150,-0.847541,0.530730,-3.334675,-5.325257
180,-0.999074,0.043022,-0.270317,-6.277368
210,-0.890028,-0.455907,2.864546,-5.592208
240,-0.548843,-0.835925,5.252275,-3.448482
270,-0.064508,-0.997917,6.270098,-0.405319


In [8]:
# One example input
(t, x0, y0, vx0, vy0) = df_input.iloc[10]

inputs_val = tf.constant([t, x0, y0, vx0, vy0])
inputs_val

<tf.Tensor: id=0, shape=(5,), dtype=float32, numpy=
array([ 0.02739726,  1.        ,  0.        , -0.        ,  6.2831855 ],
      dtype=float32)>

In [9]:
# Create a DataSet
ds = tf.data.Dataset.from_tensor_slices((dict(df_input), dict(df_output)))
ds

<TensorSliceDataset shapes: ({t: (), x0: (), y0: (), vx0: (), vy0: ()}, {x: (), y: (), vx: (), vy: ()}), types: ({t: tf.float64, x0: tf.float64, y0: tf.float64, vx0: tf.float64, vy0: tf.float64}, {x: tf.float64, y: tf.float64, vx: tf.float64, vy: tf.float64})>

In [10]:
# Example batch of inputs and outputs
for batch_in, batch_out in ds.take(1):
    pass

In [11]:
batch_in

{'t': <tf.Tensor: id=20, shape=(), dtype=float64, numpy=0.0>,
 'x0': <tf.Tensor: id=23, shape=(), dtype=float64, numpy=1.0>,
 'y0': <tf.Tensor: id=24, shape=(), dtype=float64, numpy=0.0>,
 'vx0': <tf.Tensor: id=21, shape=(), dtype=float64, numpy=-0.0>,
 'vy0': <tf.Tensor: id=22, shape=(), dtype=float64, numpy=6.283185307179586>}

In [12]:
batch_out

{'x': <tf.Tensor: id=27, shape=(), dtype=float64, numpy=1.0>,
 'y': <tf.Tensor: id=28, shape=(), dtype=float64, numpy=0.0>,
 'vx': <tf.Tensor: id=25, shape=(), dtype=float64, numpy=-0.0>,
 'vy': <tf.Tensor: id=26, shape=(), dtype=float64, numpy=6.283185307179586>}

In [13]:
batch_in['t']

<tf.Tensor: id=20, shape=(), dtype=float64, numpy=0.0>

In [14]:
# Collection of feature columns - inputs
columns_in = [tf.feature_column.numeric_column(col_name) for col_name in ['t', 'x0', 'y0', 'vx0', 'vy0']]
columns_in

[NumericColumn(key='t', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='x0', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='y0', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='vx0', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='vy0', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [15]:
# Collection of feature columns - output
columns_out = [tf.feature_column.numeric_column(col_name) for col_name in ['x', 'y', 'vx', 'vy']]
columns_out

[NumericColumn(key='x', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='y', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='vx', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='vy', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [16]:
# Input layer of DenseFeatures type
inputs_fc = tf.keras.layers.DenseFeatures(columns_in)
inputs_fc

In [46]:
# Create an input layer (vanilla Tensor)
inputs = keras.Input(shape=(5,), name='inputs')
inputs

<tf.Tensor 'inputs_1:0' shape=(None, 5) dtype=float32>

In [47]:
t = inputs[:, 0]
x0 = inputs[:, 1]
y0 = inputs[:, 2]
vx0 = inputs[:, 3]
vy0 = inputs[:, 4]

In [50]:
# Unpack variables from input layer
t = tf.Variable(lambda : inputs[:, 0])
x0 = tf.Variable(lambda : inputs[:, 1])
y0 = tf.Variable(lambda : inputs[:, 2])
vx0 = tf.Variable(lambda : inputs[:, 3])
vy0 = tf.Variable(lambda : inputs[:, 4])

In [51]:
# The semi-major axis
a = tf.Variable(1.0, name='a')

# The eccentricity
e = tf.Variable(0.0, name='e')

# The angular frequency omega
omega = tf.Variable(omega_earth, name='omega')

# The angle theta (true anomaly)
theta = omega * t

# The semi-latus rectum
p = a * (1.0 - e*e)

# The radius
r = p / (1 + e * tf.cos(theta))

# The x and y position
x = r * tf.cos(theta)
y = r * tf.sin(theta)

# The x and y velocity
#TODO: update this later with automatic differentiation!
vx = omega * r * tf.cos(theta)
vy = omega * r * tf.sin(theta)

FailedPreconditionError: Error while reading resource variable _AnonymousVar17 from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/_AnonymousVar17/class tensorflow::Var does not exist. [Op:ReadVariableOp]

In [32]:
# Outputs
outputs = tf.Variable([x, y, vx, vy])
outputs

<tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([0.98522013, 0.17129315, 6.190321  , 1.0762666 ], dtype=float32)>

In [49]:
# Wrap model
# model = tf.keras.models.Model()
model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

AttributeError: Tensor.op is meaningless when eager execution is enabled.

In [37]:
model(inputs_val)

NotImplementedError: When subclassing the `Model` class, you should implement a `call` method.